# IoU

To actually get the result we want we just need to call the metric_calc function,giving it as input the list of boxes (result of DETR) and the address of the heatmap

In [1]:
import cv2
import numpy as np

def metric_calc(box_list, img_address, tau):
    
    img = cv2.imread(img_address)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_norm = gray_norm / gray_norm.max()

    ### thresholded image
    _, thresh = cv2.threshold(gray_norm, tau, 1, cv2.THRESH_BINARY)
    
    tmp_box = boxinator(box_list.tolist(), img)
    tmp_img = thresh.astype(bool)
    max_IoU = 0
    max_IoU_ID = 0
    
    for i in tmp_box:
        overlap = i * tmp_img 
        union = i + tmp_img 
        union[union > 1] = 1

        IOU = overlap.sum() / float(union.sum())

        if IOU > max_IoU:
            max_IoU=IOU
            max_IoU_ID=i
    
    return max_IoU

def boxinator(boxes, image): 

    boxes = np.around(boxes).astype(int)

    box = np.array([image*0]*len(boxes), dtype=bool) # 3D tensor

    for i in range(len(boxes)):
        #now we are looking at box i

        box[i][np.array(boxes[i][0]):np.array(boxes[i][2])][np.array(boxes[i][1]):np.array(boxes[i][3])] = 1
        
    return box   

